# Text-to-Image Generation with NoobAI-XL-1.1
This notebook demonstrates how to use the NoobAI-XL-1.1 model from Hugging Face for text-to-image generation with various customizable parameters.


In [1]:
import torch
from diffusers import StableDiffusionPipeline
from huggingface_hub import login
import json

# Login to Hugging Face
login(token='YOUR_HUGGING_FACE_TOKEN')

# Load the model
model_id = 'Laxhar/noobai-XL-1.1'
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to('cuda')

# Function to update tokenizer config
def remove_tokenizer_limit(tokenizer_path):
    with open(tokenizer_path, 'r') as file:
        tokenizer_config = json.load(file)
    tokenizer_config.pop('max_length', None)
    with open(tokenizer_path, 'w') as file:
        json.dump(tokenizer_config, file)

# Update tokenizer configs
remove_tokenizer_limit('tokenizer/tokenizer_config.json')
remove_tokenizer_limit('tokenizer_2/tokenizer_config.json')


## Generate images with customizable parameters

In [2]:
from PIL import Image

def generate_image(prompt, sampler='pndm', steps=50, seed=42, cfg_scale=7.5, image_size=(512, 512), hires_fix=False):
    generator = torch.manual_seed(seed)
    if hires_fix:
        pipe.scheduler = pipe.schedulers[sampler](pipe.model)
        images = pipe(prompt, num_inference_steps=steps, guidance_scale=cfg_scale, generator=generator, height=image_size[0], width=image_size[1], hires_fix=True).images
    else:
        pipe.scheduler = pipe.schedulers[sampler](pipe.model)
        images = pipe(prompt, num_inference_steps=steps, guidance_scale=cfg_scale, generator=generator, height=image_size[0], width=image_size[1]).images
    return images[0]

# Example usage
prompt = 'A fantasy landscape with castles and dragons'
image = generate_image(prompt, sampler='ddim', steps=100, seed=123, cfg_scale=10.0, image_size=(768, 768), hires_fix=True)
image.show()